# Graph Generation

This notebook generates a series of gephi graph files displaying different aspects of a literary text marked up using the Chronotopic Cartographies XML schema.

**To generate the graphs:**
- Put your **valid** XML file in the 'files/xml/' folder, in the folder containing this notebook. 
- Edit cell 3 so that the 'input_file' variable is the name of the file you want to process. 
- Run the cells in order - the output from cell 5 will give you warnings if your xml file isn't valid, and will tell you what lines the problems are on. Edit the XML file and re-run cells 4 and 5 until you don't get any further errors.
- Run cell 6. Your graphs will be saved in 'files/graphs/' and can be opened in Gephi for layout.

In [2]:
import graph_generators

In [3]:
# Edit the 'input_file' variable below to the name of your marked up XML file. 
# Make sure it's in the folder named in the 'xml_dir' variable - if you've downloaded this notebook from GitHub and are running it on your own machine, you shouldn't need to change this.

xml_dir = 'files/xml/'
input_file = 'rime-of-the-ancient-mariner_1798.xml'

In [4]:
tester = graph_generators.CCXMLValidator(file=xml_dir + input_file)

In [5]:
tester.check_xml()

Errors:
No errors found!
Attribute typos:
No attribute typos found!
Source and Target mis-matches:
No mismatches found!



In [6]:
graph_generators.generate_all(xml_dir=xml_dir, output_dir='files/graphs/', input_file=input_file)